In [1]:
import pandas as pd 

df_article = pd.read_csv("data/unfinished/train.art.shuf1000.txt", sep = "\t", names = ["article"])
df_abstract = pd.read_csv("data/unfinished/train.abs.shuf1000.txt", sep = "\t", names = ["abstract"])

In [2]:
print(len(df_article))
print(len(df_abstract))

1000
1000


In [3]:
# df_article['article'].to_list()

In [4]:
df_abstract.head()

,abstract
0,two more <unk> pan am drug tests
1,argentine basketball completes olympic prepara...
2,media watchdog concerned about gambia press trial
3,lynyrd skynyrd graves vandalized
4,cingular and verizon 's good old-fashioned ad war


In [5]:
#  bulid vocabulary
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

PAD_TOKEN = '[PAD]' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNKNOWN_TOKEN = '[UNK]' # This has a vocab id, which is used to represent out-of-vocabulary words
START_DECODING = '[START]' # This has a vocab id, which is used at the start of every decoder input sequence
STOP_DECODING = '[STOP]' # This has a vocab id, which is used at the end of untruncated target sequences

tokenizer = get_tokenizer('basic_english')

def yield_tokens(df_list):
    for df in df_list:
        for text in df:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens([df_abstract['abstract'].to_list(),df_article['article'].to_list()]), specials=['[UNK]','[PAD]','[START]','[STOP]'])
vocab.set_default_index(vocab['[UNK]'])


/home/wzm289/miniconda3/envs/ACL/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
vocab.lookup_token(0)

'[UNK]'

In [7]:
vocab.__getitem__("[START]")

2

In [8]:
from data_util import data
from data_util import config

class Example(object):
    def __init__(self, article, abstract_sentences, vocab):
        # Get ids of special tokens
        start_decoding = vocab.__getitem__(data.START_DECODING)
        stop_decoding = vocab.__getitem__(data.STOP_DECODING)

        # Process the article
        article_words = article.split()
        if len(article_words) > config.max_enc_steps:
            article_words = article_words[: config.max_enc_steps]
        self.enc_len = len(
            article_words
        )  # store the length after truncation but before padding
        self.enc_input = [
            vocab.__getitem__(w) for w in article_words
        ]  # list of word ids; OOVs are represented by the id for UNK token

        # Process the abstract
        abstract = " ".join(abstract_sentences)  # string
        abstract_words = abstract.split()  # list of strings
        abs_ids = [
            vocab.__getitem__(w) for w in abstract_words
        ]  # list of word ids; OOVs are represented by the id for UNK token

        # Get the decoder input sequence and target sequence
        self.dec_input, _ = self.get_dec_inp_targ_seqs(
            abs_ids, config.max_dec_steps, start_decoding, stop_decoding
        )
        self.dec_len = len(self.dec_input)

        # If using pointer-generator mode, we need to store some extra info
        # Store a version of the enc_input where in-article OOVs are represented by their temporary OOV id; also store the in-article OOVs words themselves
        self.enc_input_extend_vocab, self.article_oovs = data.article2ids_new(
            article_words, vocab
        )

        # Get a verison of the reference summary where in-article OOVs are represented by their temporary article OOV id
        abs_ids_extend_vocab = data.abstract2ids_new(
            abstract_words, vocab, self.article_oovs
        )

        # Get decoder target sequence
        _, self.target = self.get_dec_inp_targ_seqs(
            abs_ids_extend_vocab, config.max_dec_steps, start_decoding, stop_decoding
        )

        # Store the original strings
        self.original_article = article
        self.original_abstract = abstract
        self.original_abstract_sents = abstract_sentences

    def get_dec_inp_targ_seqs(self, sequence, max_len, start_id, stop_id):
        inp = [start_id] + sequence[:]
        target = sequence[:]
        if len(inp) > max_len:  # truncate
            inp = inp[:max_len]
            target = target[:max_len]  # no end_token
        else:  # no truncation
            target.append(stop_id)  # end token
        assert len(inp) == len(target)
        return inp, target

    def pad_decoder_inp_targ(self, max_len, pad_id):
        while len(self.dec_input) < max_len:
            self.dec_input.append(pad_id)
        while len(self.target) < max_len:
            self.target.append(pad_id)

    def pad_encoder_input(self, max_len, pad_id):
        while len(self.enc_input) < max_len:
            self.enc_input.append(pad_id)
        while len(self.enc_input_extend_vocab) < max_len:
            self.enc_input_extend_vocab.append(pad_id)

2022-12-15 21:51:36.657484: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 21:51:38.252423: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-15 21:51:38.252566: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-15 21:51:38.252580: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [9]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class S2SDataset(Dataset):
    def __init__(self,):
        self.df_article = pd.read_csv("data/unfinished/train.art.shuf1000.txt", sep = "\t", names = ["article"])
        self.df_abstract = pd.read_csv("data/unfinished/train.abs.shuf1000.txt", sep = "\t", names = ["abstract"])
        
    def __len__(self):
        return len(self.df_abstract)
    def __getitem__(self, index):
        article = self.df_article["article"].iloc[index]
        abstract = self.df_abstract['abstract'].iloc[index]
        ex = Example(article, [abstract], vocab)
        return ex


In [10]:
# encoding the dataset

import numpy as np 

class Batch(object):
    def __init__(self, example_list, vocab, batch_size):
        self.batch_size = batch_size
        self.pad_id = vocab.__getitem__(
            data.PAD_TOKEN
        )  # id of the PAD token used to pad sequences
        self.init_encoder_seq(example_list)  # initialize the input to the encoder
        self.init_decoder_seq(
            example_list
        )  # initialize the input and targets for the decoder
        self.store_orig_strings(example_list)  # store the original strings

    def init_encoder_seq(self, example_list):
        # Determine the maximum length of the encoder input sequence in this batch
        max_enc_seq_len = max([ex.enc_len for ex in example_list])

        # Pad the encoder input sequences up to the length of the longest sequence
        for ex in example_list:
            ex.pad_encoder_input(max_enc_seq_len, self.pad_id)

        # Initialize the numpy arrays
        # Note: our enc_batch can have different length (second dimension) for each batch because we use dynamic_rnn for the encoder.
        self.enc_batch = np.zeros((self.batch_size, max_enc_seq_len), dtype=np.int32)
        self.enc_lens = np.zeros((self.batch_size), dtype=np.int32)
        self.enc_padding_mask = np.zeros(
            (self.batch_size, max_enc_seq_len), dtype=np.float32
        )

        # Fill in the numpy arrays
        for i, ex in enumerate(example_list):
            self.enc_batch[i, :] = ex.enc_input[:]
            self.enc_lens[i] = ex.enc_len
            for j in range(ex.enc_len):
                self.enc_padding_mask[i][j] = 1

        # For pointer-generator mode, need to store some extra info
        # Determine the max number of in-article OOVs in this batch
        self.max_art_oovs = max([len(ex.article_oovs) for ex in example_list])
        # Store the in-article OOVs themselves
        self.art_oovs = [ex.article_oovs for ex in example_list]
        # Store the version of the enc_batch that uses the article OOV ids
        self.enc_batch_extend_vocab = np.zeros(
            (self.batch_size, max_enc_seq_len), dtype=np.int32
        )
        for i, ex in enumerate(example_list):
            self.enc_batch_extend_vocab[i, :] = ex.enc_input_extend_vocab[:]

    def init_decoder_seq(self, example_list):
        # Pad the inputs and targets
        for ex in example_list:
            ex.pad_decoder_inp_targ(config.max_dec_steps, self.pad_id)

        # Initialize the numpy arrays.
        self.dec_batch = np.zeros(
            (self.batch_size, config.max_dec_steps), dtype=np.int32
        )
        self.target_batch = np.zeros(
            (self.batch_size, config.max_dec_steps), dtype=np.int32
        )
        # self.dec_padding_mask = np.zeros((self.batch_size, config.max_dec_steps), dtype=np.float32)
        self.dec_lens = np.zeros((self.batch_size), dtype=np.int32)

        # Fill in the numpy arrays
        for i, ex in enumerate(example_list):
            self.dec_batch[i, :] = ex.dec_input[:]
            self.target_batch[i, :] = ex.target[:]
            self.dec_lens[i] = ex.dec_len
            # for j in range(ex.dec_len):
            #   self.dec_padding_mask[i][j] = 1

    def store_orig_strings(self, example_list):
        self.original_articles = [
            ex.original_article for ex in example_list
        ]  # list of lists
        self.original_abstracts = [
            ex.original_abstract for ex in example_list
        ]  # list of lists
        self.original_abstracts_sents = [
            ex.original_abstract_sents for ex in example_list
        ]  # list of list of lists

def collate_batch(batch):

    return Batch(batch, vocab, len(batch))
    
    # we need to encoding the d

    # we need to padding hte dataset for the max length
    


In [11]:
train_dataset = S2SDataset()
train_dataloader = DataLoader(train_dataset, collate_fn=collate_batch, batch_size=64)

In [12]:
batch = next(iter(train_dataloader))

In [13]:
import torch as T
def get_cuda(tensor):
    if T.cuda.is_available():
        tensor = tensor.cuda()
    return tensor

def get_enc_data(batch):
    batch_size = len(batch.enc_lens)
    enc_batch = T.from_numpy(batch.enc_batch).long()
    enc_padding_mask = T.from_numpy(batch.enc_padding_mask).float()

    enc_lens = batch.enc_lens

    ct_e = T.zeros(batch_size, 2*config.hidden_dim)

    enc_batch = get_cuda(enc_batch)
    enc_padding_mask = get_cuda(enc_padding_mask)

    ct_e = get_cuda(ct_e)

    enc_batch_extend_vocab = None
    if batch.enc_batch_extend_vocab is not None:
        enc_batch_extend_vocab = T.from_numpy(batch.enc_batch_extend_vocab).long()
        enc_batch_extend_vocab = get_cuda(enc_batch_extend_vocab)

    extra_zeros = None
    if batch.max_art_oovs > 0:
        extra_zeros = T.zeros(batch_size, batch.max_art_oovs)
        extra_zeros = get_cuda(extra_zeros)


    return enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, ct_e


def get_dec_data(batch):
    dec_batch = T.from_numpy(batch.dec_batch).long()
    dec_lens = batch.dec_lens
    max_dec_len = np.max(dec_lens)
    dec_lens = T.from_numpy(batch.dec_lens).float()

    target_batch = T.from_numpy(batch.target_batch).long()

    dec_batch = get_cuda(dec_batch)
    dec_lens = get_cuda(dec_lens)
    target_batch = get_cuda(target_batch)

    return dec_batch, max_dec_len, dec_lens, target_batch

In [14]:
dec_batch, max_dec_len, dec_lens, target_batch = get_dec_data(batch)

In [15]:
dec_batch.shape

torch.Size([64, 15])

In [16]:
max_dec_len

15

In [17]:
dec_lens

tensor([ 8.,  8.,  8.,  5.,  9.,  6.,  7.,  9.,  8.,  5.,  6., 11.,  7., 11.,
        12., 10.,  7.,  8., 14., 14.,  7.,  7.,  8.,  8., 11.,  8.,  8., 13.,
         7.,  5., 10.,  9.,  7.,  9., 12.,  5.,  9.,  8., 12., 10.,  8.,  9.,
         9., 15.,  9., 12.,  6.,  9.,  7.,  8., 11.,  8.,  8., 11., 12., 15.,
         7.,  8.,  7.,  8.,  9.,  8.,  9., 13.], device='cuda:0')

In [18]:
target_batch.size()

torch.Size([64, 15])

# model

In [21]:
import pytorch_lightning as pl
import torch.nn.functional as F
import torch
from model import Model

class pl_model(pl.LightningModule):

    def __init__(self) -> None:
        super().__init__()

        # get the model 
        self.model = Model()
        self.vocab = vocab
        self.start_id = self.vocab.__getitem__(data.START_DECODING)
        self.end_id = self.vocab.__getitem__(data.STOP_DECODING)
        self.pad_id = self.vocab.__getitem__(data.PAD_TOKEN)
        self.unk_id = self.vocab.__getitem__(data.UNKNOWN_TOKEN)
    def forward(self, enc_batch, enc_lens):
        enc_batch = self.model.embeds(enc_batch)  # Get embeddings for encoder input
        enc_out, enc_hidden = self.model.encoder(enc_batch, enc_lens)

        return enc_out, enc_hidden
        
    def training_step(self, batch, batch_idx):

        (
            enc_batch,
            enc_lens,
            enc_padding_mask,
            enc_batch_extend_vocab,
            extra_zeros,
            context,
        ) = get_enc_data(batch)
        
        enc_out, enc_hidden = self(enc_batch, enc_lens)
        dec_batch, max_dec_len, dec_lens, target_batch = get_dec_data(batch)

        # Get input and target batchs for training decoder
        step_losses = []
        s_t = (enc_hidden[0], enc_hidden[1])  # Decoder hidden states
        x_t = get_cuda(
            T.LongTensor(len(enc_out)).fill_(self.start_id)
        )  # Input to the decoder
        prev_s = None  # Used for intra-decoder attention (section 2.2 in https://arxiv.org/pdf/1705.04304.pdf)
        sum_temporal_srcs = None  # Used for intra-temporal attention (section 2.1 in https://arxiv.org/pdf/1705.04304.pdf)
        for t in range(min(max_dec_len, config.max_dec_steps)):
            use_gound_truth = get_cuda(
                (T.rand(len(enc_out)) > 0.25)
            ).long()  # Probabilities indicating whether to use ground truth labels instead of previous decoded tokens
            x_t = (
                use_gound_truth * dec_batch[:, t] + (1 - use_gound_truth) * x_t
            )  # Select decoder input based on use_ground_truth probabilities
            x_t = self.model.embeds(x_t)
            final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = self.model.decoder(
                x_t,
                s_t,
                enc_out,
                enc_padding_mask,
                context,
                extra_zeros,
                enc_batch_extend_vocab,
                sum_temporal_srcs,
                prev_s,
            )
            target = target_batch[:, t]
            log_probs = T.log(final_dist + config.eps)
            step_loss = F.nll_loss(
                log_probs, target, reduction="none", ignore_index=self.pad_id
            )
            step_losses.append(step_loss)
            x_t = T.multinomial(
                final_dist, 1
            ).squeeze()  # Sample words from final distribution which can be used as input in next time step
            is_oov = (
                x_t >= config.vocab_size
            ).long()  # Mask indicating whether sampled word is OOV
            x_t = (1 - is_oov) * x_t.detach() + (
                is_oov
            ) * self.unk_id  # Replace OOVs with [UNK] token

        losses = T.sum(
            T.stack(step_losses, 1), 1
        )  # unnormalized losses for each example in the batch; (batch_size)
        batch_avg_loss = losses / dec_lens  # Normalized losses; (batch_size)
        mle_loss = T.mean(batch_avg_loss)  # Average batch loss
        return mle_loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.02)
        

In [22]:
from pytorch_lightning import Trainer
pl_model = pl_model()

trainer = Trainer(accelerator="gpu", devices=1)
trainer.fit(pl_model, train_dataloaders=train_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /maps/projects/futhark1/data/wzm289/code/Text-Summarizer-Pytorch/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params
--------------------------------
0 | model | Model | 48.2 M
1 | vocab | Vocab | 0     
--------------------------------
48.2 M    Trainable params
0         Non-trainable params
48.2 M    Total params
192.939   Total estimated model params size (MB)


Epoch 11: 100%|██████████| 16/16 [00:02<00:00,  7.66it/s, loss=15.7, v_num=0]

/home/wzm289/miniconda3/envs/ACL/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
